In [119]:
import numpy as np
import pandas as pd 
import geopandas as gpd
import matplotlib.pyplot as plt
from census import Census
from us import states
from functools import reduce
import statsmodels.api as sm
from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit

In [65]:
c = Census('205e4ff6cd960a68fc9a37a90cb79af0cddb1849')

In [66]:
# Pull Data from Census API
hills_pop19= pd.DataFrame(c.acs5.state_county_blockgroup(("NAME","GEO_ID", "B01001_001E"),
                                          states.FL.fips, "057", Census.ALL, year=2019))
hills_pop18= pd.DataFrame(c.acs5.state_county_blockgroup(("NAME","GEO_ID", "B01001_001E"),
                                          states.FL.fips, "057", Census.ALL, year=2018))
hills_pop17= pd.DataFrame(c.acs5.state_county_blockgroup(("NAME","GEO_ID", "B01001_001E"),
                                          states.FL.fips, "057", Census.ALL, year=2017))
hills_pop16= pd.DataFrame(c.acs5.state_county_blockgroup(("NAME","GEO_ID", "B01001_001E"),
                                          states.FL.fips, "057", Census.ALL, year=2016))
hills_pop15= pd.DataFrame(c.acs5.state_county_blockgroup(("NAME","GEO_ID", "B01001_001E"),
                                          states.FL.fips, "057", Census.ALL, year=2015))
hills_pop14= pd.DataFrame(c.acs5.state_county_blockgroup(("NAME","GEO_ID", "B01001_001E"),
                                          states.FL.fips, "057", Census.ALL, year=2014))
hills_pop13= pd.DataFrame(c.acs5.state_county_blockgroup(("NAME","GEO_ID", "B01001_001E"),
                                          states.FL.fips, "057", Census.ALL, year=2013))

In [67]:
# hills_pop12= c.acs5.state_county_blockgroup(("NAME", "B01001_001E"),
#                                           states.FL.fips, "057", Census.ALL, year=2012)
# hills_pop11= c.acs5.state_county_blockgroup(("NAME", "B01001_001E"),
#                                           states.FL.fips, "057", Census.ALL, year=2011)
# hills_pop10= c.acs5.state_county_blockgroup(("NAME", "B01001_001E"),
#                                           states.FL.fips, "057", Census.ALL, year=2010)
# hills_pop09= c.acs5.state_county_blockgroup(("NAME", "B01001_001E"),
#                                           states.FL.fips, "057", Census.ALL, year=2009)

In [68]:
# def get_pop(year):
#     pop_data = (c.acs5.state_county_blockgroup(("NAME", "B01001_001E"),
#                                           states.FL.fips, "057", Census.ALL, year=year))
#     pop_df = pd.DataFrame(pop_data)
#     return(pop_df)

In [69]:
#Merge all the dataframes together
data_frames=[hills_pop19,hills_pop18,hills_pop17,hills_pop16,
           hills_pop15,hills_pop14,hills_pop13]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['NAME',"state","county",
                                                               'tract','block group', "GEO_ID"],
                                            how='outer'), data_frames)

In [70]:
# yr_list = [2019,2018,2017,2016,2015,2014,2013]
# name_list = []
# for yr in yr_list:
#     name_list.append("pop_df_{}".format(str(yr)[-2:]))
# for name, yr in zip(name_list, yr_list):
#     name = get_pop(year)

In [72]:
#df_merged.columns
# Rename colums to add suffix indicating year after population table
df_merged.columns = ['NAME', 'GEO_ID', 'B01001_001E_19', 'state', 'county', 'tract', 'block group',
       'B01001_001E_18', 'B01001_001E_17', 'B01001_001E_16', 'B01001_001E_15',
       'B01001_001E_14', 'B01001_001E_13']


In [73]:
# Rearrange column order 
df_merged = df_merged[['NAME', "GEO_ID", 'state', 'county', 'tract', 'block group',
                       'B01001_001E_13','B01001_001E_14', 'B01001_001E_15', 
                       'B01001_001E_16', 'B01001_001E_17', 'B01001_001E_18',
                       'B01001_001E_19'
                      ]]

In [74]:
#Split GEO_ID to extract section that matches block group GEOID 
df_merged['GEOID'] = df_merged["GEO_ID"].apply(lambda x: x.split("US")[1])

In [75]:
#Read in State of Florida blockgroup tiger shapefile from Census
fl_bgs = gpd.read_file("/Users/jt/Documents/GIS/General/FL_Block_Groups/tl_2019_12_bg/tl_2019_12_bg.shp")

In [76]:
#Merge blockgroup and populaiton dataframes dropping null values and resulting in data from 
#Hillsborough county only
hills_bgs = fl_bgs.merge(df_merged,how="left",on="GEOID").dropna()

In [138]:
# Save hillsboroug county df
#hills_bgs.to_excel('hillsborough_county_block_group_populaiton_2013-2019.xlsx')

In [78]:
hills_bgs.reset_index(drop=True,inplace=True)

In [79]:
# hills_bgs['Combined_Col'] = hills_bgs.iloc[:,-7:].apply(lambda x: ",".join(x.astype(str)),
#                                                         axis=1)
# hills_bgs['Combined_Col']

In [80]:
# hills_bgs[["B01001_001E_19","B01001_001E_18","B01001_001E_17",
#            "B01001_001E_16","B01001_001E_15","B01001_001E_14",
#            "B01001_001E_13"]]

In [46]:
# Set index of hills df to GEOID and transpose just the populaiton data to get 
# blockgroup code as column header.
hills_bgs.set_index("GEOID",inplace=True)
year_index = [[2013,2014,2015,2016,2017,2018,2019]]
pop_data = hills_bgs.iloc[:,-7:].transpose()
#Set new index as ACS year and reset index to make column (Could have been done in one step)
pop_data.set_index(year_index, inplace=True)
pop_data.reset_index(inplace=True)

In [50]:
# Get get the colums make list and insert year to change "index" header to "year"
cols = pop_data.columns
col_list = list(cols[1:])
col_list.insert(0,'year')
pop_data.columns = [col_list]

In [112]:
# Testing regression method for analysis 
X = pop_data["year"]
y = pop_data.iloc[:,2]
y = pd.DataFrame(y)

In [131]:
# Testing regression method for analysis 
reg = LinearRegression().fit(X,y)

In [133]:
# Testing regression method for analysis 


X_pred = pd.DataFrame({'year':[2030,2040,2050,2060,2070,2080,2090,2100]})
y_pred = reg.predict(X_pred)
y_pred